# High-Dimensional Linear Models: Overfitting Simulation (Julia)

This notebook demonstrates the overfitting phenomenon in high-dimensional linear models using Julia.
We'll generate data with a nonlinear relationship and fit linear models with increasing numbers of polynomial features.

In [ ]:
using Random, Distributions, LinearAlgebra
using DataFrames, CSV
using Plots, StatsPlots
using GLM, StatsBase
using MLBase

# Set random seed for reproducibility
Random.seed!(42)
println("Packages loaded successfully")

## Data Generating Process

We use the specified data generating process:
- f_X = exp(4 * X) - 1
- y = f_X + ε, where ε ~ N(0, σ²)
- n = 1000 observations
- Intercept = 0

In [ ]:
# Parameters
n = 1000
noise_std = 0.5  # Standard deviation of the error term

# Generate X from uniform distribution
X = rand(Uniform(-0.5, 0.5), n)

# Data generating process: f_X = exp(4 * X) - 1
f_X = exp.(4 * X) .- 1

# Add noise to get Y
epsilon = rand(Normal(0, noise_std), n)
Y = f_X + epsilon

println("Generated $n observations")
println("X range: [$(minimum(X)), $(maximum(X))]")
println("Y range: [$(minimum(Y)), $(maximum(Y))]")
println("True function range: [$(minimum(f_X)), $(maximum(f_X))]")

## Visualization of the True Relationship

In [ ]:
# Plot the true relationship
X_sorted = sort(X)
f_sorted = exp.(4 * X_sorted) .- 1

p1 = scatter(X, Y, alpha=0.5, ms=2, label="Observed data", 
            xlabel="X", ylabel="Y", title="Data Generating Process")
plot!(p1, X_sorted, f_sorted, linewidth=2, color=:red, 
      label="True function: exp(4X) - 1")
display(p1)

## Functions for Model Fitting and Evaluation

In [ ]:
function create_polynomial_features(x, degree)
    """Create polynomial features up to specified degree"""
    n = length(x)
    X_poly = zeros(n, degree)
    
    for i in 1:degree
        X_poly[:, i] = x .^ i
    end
    
    return X_poly
end

function calculate_r2(y_true, y_pred)
    """Calculate R-squared"""
    ss_res = sum((y_true - y_pred).^2)
    ss_tot = sum((y_true .- mean(y_true)).^2)
    return 1 - ss_res / ss_tot
end

function calculate_adjusted_r2(r2, n, p)
    """Calculate adjusted R-squared"""
    if n <= p + 1
        return NaN
    end
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)
end

function fit_and_evaluate(X_train, y_train, X_test, y_test, n_features)
    """Fit polynomial regression model and calculate metrics"""
    try
        # Create polynomial features
        X_train_poly = create_polynomial_features(X_train, n_features)
        X_test_poly = create_polynomial_features(X_test, n_features)
        
        # Fit linear regression without intercept
        # Using normal equations: β = (X'X)^(-1)X'y
        coeffs = (X_train_poly' * X_train_poly) \ (X_train_poly' * y_train)
        
        # Predictions
        y_train_pred = X_train_poly * coeffs
        y_test_pred = X_test_poly * coeffs
        
        # Calculate metrics
        r2_train = calculate_r2(y_train, y_train_pred)
        r2_test = calculate_r2(y_test, y_test_pred)
        adj_r2 = calculate_adjusted_r2(r2_train, length(y_train), n_features)
        
        return Dict(
            "r2" => r2_train,
            "adj_r2" => adj_r2,
            "r2_oos" => r2_test,
            "n_params" => n_features,
            "success" => true
        )
    catch e
        return Dict(
            "r2" => NaN,
            "adj_r2" => NaN,
            "r2_oos" => NaN,
            "n_params" => n_features,
            "success" => false,
            "error" => string(e)
        )
    end
end

println("Functions defined successfully")

## Main Simulation Loop

We'll test models with different numbers of polynomial features: 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000

In [ ]:
# Split data into train (75%) and test (25%)
train_size = Int(floor(0.75 * n))
indices = randperm(n)
train_idx = indices[1:train_size]
test_idx = indices[train_size+1:end]

X_train = X[train_idx]
y_train = Y[train_idx]
X_test = X[test_idx]
y_test = Y[test_idx]

println("Training set size: $(length(X_train))")
println("Test set size: $(length(X_test))")

# Number of features to test
feature_counts = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]

# Storage for results
results = DataFrame(
    n_features = Int[],
    r2 = Float64[],
    adj_r2 = Float64[],
    r2_oos = Float64[],
    n_params = Int[]
)

println("\nRunning simulation...")
println("Features | R² (train) | Adj R² | R² (test) | Parameters")
println("-"^60)

for n_features in feature_counts
    # Skip if we don't have enough training samples
    if n_features >= length(X_train)
        println("$(lpad(n_features, 8)) | Skipped (insufficient training data)")
        continue
    end
    
    metrics = fit_and_evaluate(X_train, y_train, X_test, y_test, n_features)
    
    if metrics["success"]
        push!(results, (
            n_features = n_features,
            r2 = metrics["r2"],
            adj_r2 = metrics["adj_r2"],
            r2_oos = metrics["r2_oos"],
            n_params = metrics["n_params"]
        ))
        
        println("$(lpad(n_features, 8)) | $(rpad(round(metrics["r2"], digits=4), 9)) | $(rpad(round(metrics["adj_r2"], digits=4), 6)) | $(rpad(round(metrics["r2_oos"], digits=4), 8)) | $(lpad(metrics["n_params"], 9))")
    else
        println("$(lpad(n_features, 8)) | Error: $(metrics["error"][1:min(30, end)])...")
    end
end

println("\nCompleted simulation with $(nrow(results)) successful models")

## Results Visualization

We create three separate plots showing how the different R-squared metrics change with the number of features.

In [ ]:
# Create the three plots
p1 = plot(results.n_features, results.r2, 
         marker=:circle, linewidth=2, markersize=6,
         xlabel="Number of Features", ylabel="R-squared (Training)",
         title="Training R-squared vs Number of Features",
         xscale=:log10, ylims=(0, 1), grid=true, color=:blue)

# Filter out NaN values for adjusted R²
valid_adj = filter(row -> !isnan(row.adj_r2), results)
p2 = plot(valid_adj.n_features, valid_adj.adj_r2, 
         marker=:circle, linewidth=2, markersize=6,
         xlabel="Number of Features", ylabel="Adjusted R-squared",
         title="Adjusted R-squared vs Number of Features",
         xscale=:log10, grid=true, color=:green)

p3 = plot(results.n_features, results.r2_oos, 
         marker=:circle, linewidth=2, markersize=6,
         xlabel="Number of Features", ylabel="Out-of-sample R-squared",
         title="Out-of-sample R-squared vs Number of Features",
         xscale=:log10, grid=true, color=:red)

# Combine plots
combined_plot = plot(p1, p2, p3, layout=(1,3), size=(1200, 400))
display(combined_plot)

## Summary and Analysis

Let's examine the results and discuss the overfitting phenomenon.

In [ ]:
# Display summary statistics
println("Summary of Results:")
println("="^80)
show(results, allrows=true)

# Find optimal number of features based on out-of-sample R²
best_idx = argmax(results.r2_oos)
best_n_features = results.n_features[best_idx]
best_oos_r2 = results.r2_oos[best_idx]

println("\n\nOptimal number of features (based on out-of-sample R²): $best_n_features")
println("Best out-of-sample R²: $(round(best_oos_r2, digits=4))")

# Calculate the difference between training and test R² to show overfitting
results.overfitting = results.r2 - results.r2_oos
println("\nOverfitting Analysis (Training R² - Test R²):")
for i in 1:nrow(results)
    println("$(lpad(results.n_features[i], 8)) | $(rpad(round(results.overfitting[i], digits=4), 8))")
end

## Interpretation and Conclusions

### Overfitting Demonstration

This simulation clearly demonstrates the overfitting phenomenon in high-dimensional linear models:

1. **Training R-squared** monotonically increases as we add more polynomial features. This makes sense because with more parameters, the model can fit the training data more closely.

2. **Adjusted R-squared** initially increases but then starts to decrease as the penalty for additional parameters outweighs the improvement in fit. This metric tries to balance model fit with model complexity.

3. **Out-of-sample R-squared** typically increases initially as we capture more of the true nonlinear relationship, but then decreases as the model becomes too complex and starts fitting noise rather than signal.

### Key Insights:

- **Bias-Variance Tradeoff**: Simple models (few features) have high bias but low variance. Complex models (many features) have low bias but high variance.
- **Optimal Complexity**: There's an optimal number of features that maximizes out-of-sample performance.
- **Generalization**: Models that perform well on training data don't necessarily generalize well to new data.

### Julia-Specific Observations:

- Julia's linear algebra capabilities make matrix operations efficient
- The language's mathematical syntax closely matches the theoretical formulations
- Broadcasting operations (`.` syntax) make vectorized computations intuitive

### Practical Implications:

- Always use cross-validation or hold-out samples to evaluate model performance
- Consider regularization techniques for high-dimensional problems
- Be cautious of models with very high training accuracy but poor test performance
- The true data generating process is nonlinear (exponential), but we're using polynomial approximations